In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Constante universal dos gases
R = 8.3144598 #L.KPa/mol.K

# Parametros da EoS SRK
OMEGA = 0.08664
PSI = 0.42748 

In [ ]:
#Os índices a e w correspondem às iniciais dos componentes acetone e water

#Parâmetros UNIQUAC obtidos no Prausnitz
z = 10

#O parâmetro Qlin corresponde ao q' utilizado para a água e álcools de cadeia curta
# Acetona(1)
R1 = 2.57
Q1 = 2.34
Qlin1 = Q1
coef_antoine1 = [14.3145, 2756.22, 228.060]

#Água(2)
R2 = 0.92
Q2 = 1.400
Qlin2 = 1.00
coef_antoine2 = [16.3872, 3885.70, 230.170]

#Parâmetros de interação UNIQUAC obtidos no Prausnitz
A12 = 530.99
A21 = -100.71

In [ ]:
#Calculo da pressão de saturação via Antoine, em KPa, usada como chute inicial para o calculo de Psat e phi sat via SRK
def antoine(coef_antoine,T):
    P_sat_ant = np.exp(coef_antoine[0] - coef_antoine[1]/((T - 273.15) + coef_antoine[2]))
    return(P_sat_ant)

In [ ]:
def phi_sat(P_sat1, P_sat2,T):
    
    # Acetone
    T_crit1 = 508.2 #K
    T_r1 = T/T_crit1
    P_crit1 = 4.701*(10**3) #KPa
    ACENTRIC_FACTOR1 = 0.307
    B0_1 = 0.083 - 0.442/(T_r1**1.6)
    B1_1 = 0.139 - 0.172/(T_r1**4.2)
    
    # Water
    T_crit2 = 647.1 #K
    P_crit2 = 2.2055*(10**4) #KPa
    ACENTRIC_FACTOR2 = 0.345
    T_r2 = T/T_crit2
    B0_2 = 0.083 - 0.442/(T_r2**1.6)
    B1_2 = 0.139 - 0.172/(T_r2**4.2)
       
    #Cálculo do segundo coeficiente do tipo virial reduzido
    Bred_11 = B0_1 + ACENTRIC_FACTOR1*B1_1
    Bred_22 = B0_2 + ACENTRIC_FACTOR2*B1_2
    
    #Cálculo do segundo coeficiente do tipo virial
    B11 = Bred_11*R*T_crit1/P_crit1
    B22 = Bred_22*R*T_crit2/P_crit2
   
    phi_sat1 = np.exp(B11*P_sat1/(R*T))
    phi_sat2 = np.exp(B22*P_sat2/(R*T))
    
    return(phi_sat1,phi_sat2)    

In [ ]:
# Funcao para calcular a pressao de saturacao
def calc_Psat(T,coef_antoine,option):
    
    # Acetone
    if (option==1):
        T_crit = 508.2 #K
        P_crit = 4.701*(10**3) #KPa
        ACENTRIC_FACTOR = 0.307
        
                
    # Water
    elif (option==2):
        P_crit = 2.2055*(10**4) #KPa
        T_crit = 647.1 #K
        ACENTRIC_FACTOR = 0.345
                 
    ERRO_MIN = 1*(10**(-5))
    ITER=0
    ITER_MAX = 1000
    erro = 1        
    n=0       
    while(erro>ERRO_MIN and ITER<ITER_MAX):
        
        
        print("antoine=",coef_antoine)
        print("T=",T)
        P = antoine(coef_antoine, T)
        print('P=',P)
        # Pressao e temperatura reduzidas
        T_r = T/T_crit
        P_r = P/P_crit

        # Parametros da EoS SRK
        alfa = (1 + (0.480 + 1.574*ACENTRIC_FACTOR - 0.176*(ACENTRIC_FACTOR**2))*(1 - (T_r**0.5)))**2
        a = (PSI*alfa*(R**2)*(T_crit**2))/P_crit
        b = (OMEGA*R*T_crit)/P_crit

        # Coeficientes da forma polinomial da EoS SRK
        coef = [0]*4
        coef[0] = P
        coef[1] = -R*T
        coef[2] = -(P*(b**2) + b*R*T - a)
        coef[3] = -a*b

        # Raizes da EoS SRK para P e T especificados
        vol = np.roots(coef)
        print('vol=',vol)
        
        # Teste para determinar se ha raizes imaginarias
        for comp_vol in vol:
            if(np.dtype('Float64')!=type(comp_vol)):
                return
        
        # Parametros para o calculo de G residual
        beta = (b*P)/(R*T)
        q = a/(b*R*T)
        
        #Liquido
        Z_liq = (min(vol))*P/(R*T)
        I_liq = np.log((Z_liq + beta)/Z_liq)
        G_res_liq = R*T*(Z_liq -1 -np.log(Z_liq - beta) - q*I_liq) #G residual do liquido
        phi_liq = np.exp(G_res_liq/(R*T))
        
        #Vapor 
        Z_vap = (max(vol))*P/(R*T)
        I_vap = np.log((Z_vap + beta)/Z_vap)
        G_res_vap = R*T*(Z_vap - 1 - np.log(Z_vap - beta) - q*I_vap) #G residual do vapor
        phi_vap = np.exp(G_res_vap/(R*T)) 
        erro = np.abs(1 - phi_liq/phi_vap) # Criterio de convergencia phi_liq/phi_vap < 1.10^-9
        ITER += 1
        P = P*(phi_liq/phi_vap)
        
    return(P,phi_vap) 

In [ ]:
#Cálculo dos coeficientes de atividade para a fase líquida
def UNIQUAC(x1,T):
    
    #Fração molar da água
    x2 = 1 -x1
    
    #parâmetros da UNIQUAC
    TAU12 = np.exp( -A12/T)
    TAU21 = np.exp( -A21/T)
    PHI1 = x1*R1/(x1*R1 + x2*R2)
    PHI2 = x2*R2/(x1*R1 + x2*R2)
    TETA1 = x1*Q1/(x1*Q1 + x2*Q2)
    TETA2 = x2*Q2/(x1*Q1 + x2*Q2)
    TETAlin1 = x1*Qlin1/(x1*Qlin1 + x2*Qlin2)
    TETAlin2 = x2*Qlin2/(x1*Qlin1 + x2*Qlin2)
    L1 = (z/2)*(R1 - Q1) - (R1 - 1)
    L2 = (z/2)*(R2 - Q2) - (R2 - 1)
    
    gamma1 = np.exp(np.log(PHI1/x1) + (z/2)*Q1*np.log(TETA1/PHI1)+ 
    PHI2*(L1 - (R1/R2)*L2) + Qlin1*(TETAlin2*(TAU21/(TETAlin1 + TETAlin2*TAU21) -
    TAU12/(TETAlin2 + TETAlin1*TAU12)) - np.log(TETAlin1 + TETAlin2*TAU21)))
    
    gamma2 = np.exp(np.log(PHI2/x2) + (z/2)*Q2*np.log(TETA2/PHI2)  
    + PHI1*(L2 - (R2/R1)*L1) + Qlin2*(TETAlin1*(TAU12/(TETAlin2 + TETAlin1*TAU12) -
    TAU21/(TETAlin1 + TETAlin2*TAU21)) - np.log(TETAlin2 + TETAlin1*TAU12)))
    
    return(gamma1,gamma2)        

In [ ]:
# Função objetivo de Z no vapor para o cálculo de Z via Newton-Raphson
def Func_obj(Z,beta,q):
    
    F = 1.0 - Z + beta - q*beta*(Z - beta)/(Z*(Z + beta))
    return(F)

In [ ]:
def calc_Z(beta,q):
    
    h = 1*(10**(-5))
    Zit = 1
    ERRO_MIN = 1*(10**(-9))
    ITER=0
    ITER_MAX = 1000
    erro = 1
    
    while(erro>ERRO_MIN and ITER<ITER_MAX):
        deriv_Func_obj = (Func_obj(Zit + h,beta,q) - Func_obj(Zit - h,beta,q))/(2*h)
        Z = Zit - Func_obj(Zit,beta,q)/deriv_Func_obj
        erro = np.abs(Z-Zit)
        ITER += 1
        Zit = Z
    return(Zit)

In [ ]:
#Cálculo dos coeficientes de fugacidade da fase gasosa
def SRK_phi(P,T,y1):
    
    #Cálculo da fração molar de comp. 2
    y2 = 1 - y1
    
    # Acetone
    T_crit1 = 508.2 #K
    P_crit1 = 4.701*(10**3) #KPa
    ACENTRIC_FACTOR1 = 0.307
    T_r1 = T/T_crit1
    P_r1 = P/P_crit1
    alfa1 = (1 + (0.480 + 1.574*ACENTRIC_FACTOR1 - 0.176*(ACENTRIC_FACTOR1**2))*(1 - (T_r1**0.5)))**2
    a1 = (PSI*alfa1*(R**2)*(T_crit1**2))/P_crit1
    b1 = (OMEGA*R*T_crit1)/P_crit1
                
    # Water
    P_crit2 = 2.2055*(10**4) #KPa
    T_crit2 = 647.1 #K
    ACENTRIC_FACTOR2 = 0.345
    T_r2 = T/T_crit2
    P_r2 = P/P_crit2
    alfa2 = (1 + (0.480 + 1.574*ACENTRIC_FACTOR2 - 0.176*(ACENTRIC_FACTOR2**2))*(1 - (T_r2**0.5)))**2
    a2 = (PSI*alfa2*(R**2)*(T_crit2**2))/P_crit2
    b2 = (OMEGA*R*T_crit2)/P_crit2
    
    #Parâmetros de interação
    a = (y1**2)*a1 + 2*y1*y2*((a1*a2)**0.5) + (y2**2)*a2
    b = y1*b1 + y2*b2
    q = a/(b*R*T)
    beta = b*P/(R*T)
    
    #Cálculo de Z via SRK por Newton-Raphson
    Z = calc_Z(beta,q)
    I = np.log((Z + beta)/Z)
    
    #Parâmetros parciais da EoS
    q_parc1 = q*((2*y1*a1 + 2*y2*((a1*a2)**0.5))/a - b1/b)
    q_parc2 = q*((2*y2*a2 + 2*y1*((a1*a2)**0.5))/a - b2/b)
    
    #Cálculo de phi
    phi1 = np.exp((b1/b)*(Z - 1) - np.log(Z - beta) - q_parc1*I)
    phi2 = np.exp((b2/b)*(Z - 1) - np.log(Z - beta) - q_parc2*I)
    
    return(phi1,phi2)    

In [ ]:
def T_sat_antoine(coef,P):
    
    Tsat = coef[1]/(coef[0] - np.log(P)) - coef[2]
    Tsat= Tsat + 273.15 #Conversão para Kelvin
    return(Tsat)

In [ ]:
def BOL_T(P,x1):
    
    #Primeira parte do cálculo. Determinação de um bom chute em T para entrar no loop
    x2 = 1 - x1
    PHI_vap1 = 1
    PHI_vap2 = 1
       
    
    #Chute de T obtido através de Tsat, via Antoine
    T_sat1 = T_sat_antoine(coef_antoine1,P)
    T_sat2 = T_sat_antoine(coef_antoine2,P)
    T0 = x1*T_sat1 + x2*T_sat2
    
    
    #Cálculo de Psat via Antoine
    P_sat1 = antoine(coef_antoine1,T0)
    P_sat2 = antoine(coef_antoine2,T0)
        
    #Cálculo de gamma via UNIQUAC
    (gamma1,gamma2) = UNIQUAC(x1,T0)
    
    #Função objetivo 1
    F0 = x1*gamma1*P_sat1/(PHI_vap1*P) + x2*gamma2*P_sat2/(PHI_vap2*P) - 1
    
    #Determinação de um novo chute para T
    P_sat2 = P/((x1*gamma1/PHI_vap1)*(P_sat1/P_sat2) + (x2*gamma2/PHI_vap2)*(P_sat2/P_sat2))
    
    #Segundo
    T = T_sat_antoine(coef_antoine2,P_sat2)
      
    #Parâmetros do laço
    ERRO_MIN = 1*(10**(-6))
    ITER=0
    ITER_MAX = 100
    erro = 1
        
    while(erro>ERRO_MIN and ITER<ITER_MAX):
        
        P_sat1 = antoine(coef_antoine1,T)
        P_sat2 = antoine(coef_antoine2,T)
                
        #Cálculo de gamma via UNIQUAC
        (gamma1,gamma2) = UNIQUAC(x1,T)
        
        y1 = x1*gamma1*P_sat1/(PHI_vap1*P)
        y2 = x2*gamma2*P_sat2/(PHI_vap2*P)
        
        (phi_sat1,phi_sat2) = phi_sat(P_sat1, P_sat2,T)
        (phi1,phi2) = SRK_phi(P,T,y1)
        
        #Desprezando o fator de Poynting
        PHI_vap1 = phi1/phi_sat1
        PHI_vap2 = phi2/phi_sat2
        
        F = x1*gamma1*P_sat1/(PHI_vap1*P) + x2*gamma2*P_sat2/(PHI_vap2*P) - 1
        T1 = T - (F/(F - F0))*(T - T0)
        erro = np.abs(T1 - T)
        T0 = T
        F0 = F
        T = T1
           
    return(T,y1)

In [ ]:
def ORV_T(P,y1):
    
    #Primeira parte do cálculo. Determinação de dois chutes em T para entrarem no loop
    y2 = 1 - y1
    PHI_vap1 = 1
    PHI_vap2 = 1
    gamma1 = 1
    gamma2 = 1
    
    #Chute de T obtido através de Tsat, via Antoine
    T_sat1 = T_sat_antoine(coef_antoine1,P)
    T_sat2 = T_sat_antoine(coef_antoine2,P)
    T0 = y1*T_sat1 + y2*T_sat2
    
    #Cálculo de Psat via Antoine
    P_sat1 = antoine(coef_antoine1,T0)
    P_sat2 = antoine(coef_antoine2,T0)
    
    F0 = y1*PHI_vap1*P/(gamma1*P_sat1) + y2*PHI_vap2*P/(gamma2*P_sat2) - 1
    
    #Segundo chute
    P_sat2 = P*((y1*PHI_vap1/gamma1)*(P_sat2/P_sat1) + (y2*PHI_vap2/gamma2))
    T = T_sat_antoine(coef_antoine2,P_sat2)
    
    #Parâmetros do laço
    ERRO_MIN = 1*(10**(-6))
    ITER=0
    ITER_MAX = 100
    erro = 1
        
    while(erro>ERRO_MIN and ITER<ITER_MAX):
        
        P_sat1 = antoine(coef_antoine1,T)
        P_sat2 = antoine(coef_antoine2,T)
        
        (phi_sat1,phi_sat2) = phi_sat(P_sat1, P_sat2,T)
        (phi1,phi2) = SRK_phi(P,T,y1)
        
        #Desprezando o fator de Poynting
        PHI_vap1 = phi1/phi_sat1
        PHI_vap2 = phi2/phi_sat2
        
        x1 = y1*PHI_vap1*P/(gamma1*P_sat1)
        x2 = y2*PHI_vap2*P/(gamma2*P_sat2)
        
        #Cálculo de gamma via UNIQUAC
        (gamma1,gamma2) = UNIQUAC(x1,T)
        
        F = y1*PHI_vap1*P/(gamma1*P_sat1) + y2*PHI_vap2*P/(gamma2*P_sat2) - 1
        T1 = T - (F/(F - F0))*(T - T0)
        erro = np.abs(T1 - T)
        T0 = T
        F0 = F
        T = T1
    
    return(T,x1)  

In [ ]:
# Acessando os arquivos .txt com os dados experimentais do equilíbrio líquido-vapor
arquivo_manipulado = open("water_acetone_P1.txt", 'r')

T_exp = []  #K
x1_exp = []   
y1_exp = []

for linha in arquivo_manipulado:
    valores = linha.split()
    T_exp.append(float(valores[0]))
    x1_exp.append(float(valores[1]))
    y1_exp.append(float(valores[2]))
    
arquivo_manipulado.close()

In [ ]:
x1_calc = []         
y1_calc = []     
Temp1_K = []
Temp2_K = []
i = 0

while i < len(T_exp):

    P = 101.33 #KPa
    x1_enter = x1_exp[i] 
    y1_enter = y1_exp[i]
    (T_c1,x1_c) = ORV_T(P,y1_enter)
    (T_c2,y1_c) = BOL_T(P,x1_enter)
    x1_calc.append(x1_c)
    y1_calc.append(y1_c)
    Temp1_K.append(T_c1)
    Temp2_K.append(T_c2)
    i=i+1

In [ ]:
# Plot
plt.figure(1) 
eixos = plt.subplots(figsize=(8,10))
plt.title('T vs x,y Water-Acetone')
plt.xlabel('x1,y1')
plt.ylabel('T(K)')
plt.plot(x1_calc, Temp1_K,label='G')
plt.plot(y1_calc, Temp2_K,label='L')
plt.scatter(x1_exp, T_exp,label='experimental')
plt.scatter(y1_exp, T_exp,label='experimental')
plt.legend(loc=0)
plt.show()

In [ ]:
def G_E_function(T,x1):
    
    x2 = 1 - x1
    (gamma1,gamma2) = UNIQUAC(x1,T)
    G_Exc = R*T*(np.log(gamma1)*x1 + np.log(gamma2)*x2)
    
    return(G_Exc)    

In [ ]:
def propriedades_de_mistura(T,x1):
    
    h = 1*(10**(-5))
    G_Excess = G_E_function(T,x1)
    H_Excess = -R*(T**2)*((G_E_function(T+h,x1)/(R*(T+h)) - G_E_function(T-h,x1)/(R*(T-h)))/(2*h))
    S_Excess = -(G_E_function(T+h,x1) - G_E_function(T-h,x1))/(2*h)
        
    return(G_Excess,H_Excess,S_Excess)

In [ ]:
# Acessando os arquivos .txt com os dados experimentais do equilíbrio líquido-vapor
arquivo_manipulado = open("water_acetone_H_Exc.txt", 'r')

T = 293.150 #K
    
H_E_exp = []   
x1_exc_exp = []
G_Exc = []
H_Exc = []
S_Exc = []
TS_Exc = []

for linha in arquivo_manipulado:
    valores = linha.split()
    H_E_exp.append(float(valores[0]))
    x1_exc_exp.append(float(valores[1]))
    #print(linha)
    #print(H_E_exp)
    #print(x1_exc_exp)
    
    
arquivo_manipulado.close()

i = 0

'''print(len(x1_exc_exp))

while i < len(x1_exc_exp):

    [G_Ex,H_Ex,S_Ex] = propriedades_de_mistura(T,x1_exc_exp[i])
    G_Exc.append(G_Ex)
    H_Exc.append(H_Ex)
    S_Exc.append(S_Ex)
    TS_Exc.append(-T*S_Ex)
    print(i)
    i+=1'''
    
#print('H_E_exp',H_E_exp)



x1_exc_axis = np.linspace(0,1,100)

while i < len(x1_exc_axis):

    [G_Ex,H_Ex,S_Ex] = propriedades_de_mistura(T,x1_exc_axis[i])
    G_Exc.append(G_Ex)
    H_Exc.append(H_Ex)
    S_Exc.append(S_Ex)
    TS_Exc.append(-T*S_Ex)
    print(i)
    i+=1


In [ ]:
plt.figure(2)
plt.title('Propriedades em Excesso')
plt.xlabel('x1')
plt.ylabel('KJ/mol')

print(len(x1_calc))
print(len(G_Exc))

plt.scatter(x1_exc_exp,H_E_exp,color ='b')
plt.plot(x1_exc_axis,G_Exc,'r')
plt.plot(x1_exc_axis,H_Exc,'b')
plt.plot(x1_exc_axis,TS_Exc,'g')
plt.show()